In [ ]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
import numpy as np
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingRegressor
from sklearn.svm import LinearSVR
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
import pandas as pd

In [ ]:
uselessFeatures = ['Utilities', 'Street', 'Heating', 'Condition2', 'MiscFeature',
       'MiscVal', 'LowQualFinSF', 'PoolArea', 'BsmtHalfBath',
       'GarageCond', '3SsnPorch', 'PoolQC', 'Electrical', 'GarageQual',
       'PavedDrive', 'RoofMatl', 'Id']

def addNewFeaturesToDF(concatDropped):
    concatDropped['YearsSinceRemodeled']  = concatDropped['YrSold'] - concatDropped['YearRemodAdd']
    concatDropped['YearsOld']  = concatDropped['YrSold'] - concatDropped['YearBuilt']
    concatDropped['TotalSF']  = concatDropped['TotalBsmtSF'] + concatDropped['1stFlrSF'] + concatDropped['2ndFlrSF'] + concatDropped['1stFlrSF']
    concatDropped['UnfinishedSF']  = (concatDropped['LowQualFinSF'] + concatDropped['BsmtUnfSF']) / concatDropped['TotalSF']
    concatDropped['FinishedSFBasement']  = 1 - (concatDropped['BsmtUnfSF']) / concatDropped['TotalBsmtSF']
    concatDropped['TotalOutsideSF']  = concatDropped['WoodDeckSF'] + concatDropped['OpenPorchSF'] + concatDropped['EnclosedPorch'] + concatDropped['3SsnPorch']+ concatDropped['ScreenPorch'] + concatDropped['PoolArea']
    concatDropped['PropertySF']  = concatDropped['TotalSF'] + concatDropped['TotalOutsideSF'] + concatDropped['MasVnrArea'] + concatDropped['LotFrontage']
    concatDropped['grassArea'] = concatDropped['PropertySF']  - concatDropped['LotArea'] 
    concatDropped['bathrooms'] = concatDropped['BsmtFullBath']  + concatDropped['BsmtHalfBath'] +concatDropped['FullBath']  + concatDropped['HalfBath'] 
    concatDropped['numberBedAndBath'] = concatDropped['BedroomAbvGr']  + concatDropped['FullBath'] 
    

def encodeEntireData(X_train):
    addNewFeaturesToDF(X_train)
    X_train = X_train.drop(uselessFeatures, axis = 1)
    cache = []
    for feature in X_train.columns.values:
        if feature == 'Type':
            continue
        else:
            label_encoder =LabelEncoder()
            X_train[feature]= label_encoder.fit_transform(X_train[feature])
            cache.append(label_encoder)
            b = np.reshape(X_train[feature].to_numpy(), (X_train.shape[0],1))
            b = b / max(abs(b))
            a = b - np.mean(b)
            #a = Normalizer().fit_transform(b)
            X_train[feature] = a
    return X_train

In [ ]:
trainingData = pd.read_csv("train.csv")
trainingData['Type'] = 'train'
testData = pd.read_csv("test.csv")
testData['Type'] = 'test'

In [ ]:
concat = pd.concat([trainingData, testData], axis = 0)
concat = concat.drop('SalePrice', axis = 1)

In [ ]:
concatProcessed = encodeEntireData(concat)

X_train = concatProcessed.loc[concatProcessed.Type == 'train']
X_train = X_train.drop('Type', axis = 1)
Y_train = trainingData.SalePrice
X_val = concatProcessed.loc[concatProcessed.Type == 'test']

corrMatrix = concatProcessed.corr(method='pearson')

In [ ]:
X_train

In [ ]:
pd.set_option('display.max_rows', None)
corrMatrix['SalePrice']

In [ ]:
pd.plotting.scatter_matrix(concat2[['Id', 'SalePrice']])

In [ ]:
#automatic outlier detection
from sklearn.ensemble import IsolationForest
clf = IsolationForest(n_estimators = 10)
clf.fit(X_train)

In [ ]:
import pylab as pl
for i in X_train.columns.values:
    fig, (ax1, ax2, ax3)=plt.subplots(1,3)
    #print(i)
    ax1.set_title(i)
    ax1.hist(X_train[i], bins = np.linspace(min(X_train[i]), max(X_train[i]), 100))

    ax3.scatter(Y_train, X_train[i])

In [ ]:
X_train2, X_test, Y_train2, Y_test = train_test_split(X_train, Y_train, test_size=0.33, random_state=42)

In [ ]:
clf2 = RandomForestRegressor(n_estimators = 1000)
clf2.fit(X_train2, Y_train2)

print(clf2.score(X_train2,Y_train2))
print(clf2.score(X_test,Y_test))
y = clf2.feature_importances_
x = np.linspace(0, len(y)-1, len(y))
fig=plt.figure()
plt.plot(x,y)

In [ ]:
Y_pred = clf2.predict(X_test)
np.sum(abs(np.divide(Y_test - Y_pred, Y_test)))/len(Y_test)

In [ ]:
data = np.array([X_train.columns.values, y]).T
sortedArr = data[data[:,1].argsort()]
print(sortedArr)

## predict log of Y

In [ ]:
a = pd.DataFrame(np.log(Y_train2))
a.hist(bins=100)

In [ ]:
clf3 = RandomForestRegressor(n_estimators = 1000)
clf3.fit(X_train2, np.log(Y_train2))

print(clf3.score(X_train2,np.log(Y_train2)))
print(clf3.score(X_test,np.log(Y_test)))
y = clf3.feature_importances_
x = np.linspace(0, len(y)-1, len(y))
fig=plt.figure()
plt.plot(x,y)

Y_pred = np.exp(clf3.predict(X_test))
print(np.sum(abs(np.divide(Y_test - Y_pred, Y_test)))/len(Y_test))

# Stacked

In [ ]:
estimators = [#('rf1', RandomForestRegressor(n_estimators=10, random_state=42)),
              #('rf2', RandomForestRegressor(n_estimators=100, random_state=42)),
              #('rf3', RandomForestRegressor(n_estimators=19, random_state=42)),
              #('rf4', RandomForestRegressor(n_estimators=4, random_state=42)),
              ('rf5', RandomForestRegressor(n_estimators=300, max_leaf_nodes = 20, random_state=42)),
              ('rf6', RandomForestRegressor(n_estimators=300, max_leaf_nodes = 25, bootstrap=False, random_state=42)),
              ('rf7', RandomForestRegressor(n_estimators=100, max_leaf_nodes = 20, random_state=42)),
              ('xgboost1', XGBRegressor(learning_rate=0.1, max_depth=10, random_state=42)),
              ('xgboost2', XGBRegressor(learning_rate=0.1, max_depth=8, random_state=42)),
              ('xgboost3', XGBRegressor(n_estimators=100, learning_rate=0.01, max_depth=6, random_state=42)),
    
              ('xgboost4', XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)),
    
              #('svr', LinearSVR(random_state=42)),
              #('svr', make_pipeline(StandardScaler(),LinearSVR(random_state=42))),
              ('nn1', MLPRegressor(hidden_layer_sizes = (1000,500,100,),random_state=42, max_iter=1000, early_stopping=True)),
              #('nn2', MLPRegressor(hidden_layer_sizes = (100,),random_state=42, max_iter=500, early_stopping=True)),
              #('nn3', MLPRegressor(hidden_layer_sizes = (10, 10,),random_state=42, max_iter=5000, early_stopping=True)),
              #('logReg', LogisticRegression()),
            # ('kmeans', make_pipeline(StandardScaler(),KMeans(random_state=42))),
              
             ]

clf = StackingRegressor(
     estimators=estimators#, final_estimator = LogisticRegression(random_state=42)
    #final_estimator=RandomForestRegressor(n_estimators=100, random_state=42)
 )

In [ ]:
clf.fit(X_train2, Y_train2)
print(clf.score(X_train2,Y_train2))
print(clf.score(X_test,Y_test))